In [1]:
############
# Packages #
############
import time as t

import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import scipy as sp

###################
# hyperparametres #
###################

# for reproductibility
seed = 0

# data dimensions
T=200
k=100
l = 1

#Z : (k,1)
#Y : (T,1)
#U : (T,l)
#X : (T,k)

# data generations

# for X:
rho=0.75

# for Beta : number of non null
s=5 #in [5,10,100]
lst_s = [5,10,100]

# for sigma2 :ratio between explained and total variance 
Ry=0.02 #in [0.02, 0.25, 0.5]
lst_Ry = [0.02, 0.25, 0.5]

# for q prior
a=1
b=1

# for R2 prior
A=1
B=1

### Initialize parameters

Firstly, let's comput the initialization of the parameters

In [33]:
def compute_X(T, k, rho):
    """Compute matrix of xt observations

    Args:
        T (int): number of observations
        k (int): number of predictors
        rho (float): Toeplitz correlation parameter

    Returns:
        np.array: dimensions T*k
    """
    cov_matrix=np.zeros((k, k))
    for i in range(k):
        for j in range(k):
            cov_matrix[i,j]=rho**np.abs(i-j)
    return np.random.multivariate_normal([0]*k, cov_matrix, T)

def compute_U(T, l, rho):
    """Compute matrix of xt observations

    Args:
        T (int): number of observations
        l (int): number of predictors
        rho (float): Toeplitz correlation parameter

    Returns:
        np.array: dimensions T*l
    """
    cov_matrix=np.zeros((l, l))
    for i in range(l):
        for j in range(l):
            cov_matrix[i,j]=rho**np.abs(i-j)
    return np.random.multivariate_normal([0]*l, cov_matrix, T)

def compute_vx(X):
    return np.mean(np.var(X,axis=0))

def sample_beta(k, s):
    """Sample of beta vector of dimensions 1*k

    Args:
        k (int): number of predictors
        s (int): number of non-zero elements of beta

    Returns:
        np.array: dimensions 1*k
    """
    beta=np.zeros(k)
    index_normal_distribution=np.random.choice(len(beta), size=s, replace=False)
    beta[index_normal_distribution] = np.random.normal(loc=0, scale=1, size=s)
    return beta

def sample_phi(l):
    return np.random.uniform(0,1, size=l)

def comput_Z(beta):
    """Compute z_1,...,z_k

    Args:
        beta (np.array): random vector beta

    Returns:
        np.array: dimensions1*k
    """
    Z=beta
    Z[Z!=0]=1
    return Z

def compute_sigma2(Ry, beta, X):
    """ Compute sigma2
    Args:
        Ry (float): pourcentage of explained variance
        beta (np.array): beta previously sampled
        X (np.array): matrix of (xt) samples

    Returns:
        float: dimensions 1*1
    """
    return (1/Ry-1)*np.mean(np.square(X @ beta))

def compute_R2(q, k, gamma2, v_x):
    return (q*k*gamma2*v_x)/(q*k*gamma2*v_x+1)

def sample_epsilon(T, sigma2):
    """Sample epsilon_1,...,epsilon_T

    Args:
        T (int): number of observations
        sigma2 (float): sigma2 previously sampled

    Returns:
        np.array: dimensions 1*T
    """
    return np.random.normal(loc=0, scale=sigma2, size=T)

def compute_Y(X, beta, epsilon):
    return X@beta + epsilon


### Final function
def init_parameters(seed, T, k, l, rho, s, Ry, a, b, A, B):
    """
    Initialize parameters for a given simulation.

    Args:
        seed (int): Seed for reproducibility.
        T (int): Number of observations.
        k (int): Number of covariates.
        l (int): Number of latent variables.
        rho (float): Correlation parameter.
        s (float): Scaling parameter.
        Ry (float): Response variance.
        a (float): Shape parameter for gamma2.
        b (float): Shape parameter for gamma2.
        A (float): Shape parameter for q.
        B (float): Shape parameter for q.

    Returns:
        dict: Dictionary containing initialized parameters.
    """
    np.random.seed(seed=seed)
    dct = {
        "X" : compute_X(T=T, k=k, rho=rho),
        "U": compute_U(T=T, l=l, rho=rho),
        "beta": sample_beta(k=k, s=s),
        "phi": sample_phi(l=l),
        "q": np.random.beta(A,B),
        "gamma2": np.random.beta(a,b),
    }
    dct["vx"] = compute_vx(X=dct["X"])
    dct["R2"] = compute_R2(q=dct["q"], k=k, gamma2=dct["gamma2"], v_x=dct["vx"])
    dct["Z"]=comput_Z(beta=dct["beta"])
    dct["sigma2"] = compute_sigma2(Ry=Ry, beta=dct["beta"], X=dct["X"])
    dct["epsilon"] = sample_epsilon(T=T, sigma2=dct["sigma2"])
    dct["Y"]=compute_Y(X=dct["X"], beta=dct["beta"], epsilon=dct["epsilon"])
    return dct

In [34]:
dct = init_parameters(seed,T,k,l,rho,s,Ry,a,b,A,B)

for scalar in ["q", "gamma2", "vx", "R2", "sigma2"]:
    value = dct[scalar]
    print(f"{scalar}={value}\n")

for mat in ["beta", "phi", "Y", "epsilon", "X", "U"]:
    value = dct[mat]
    fig = px.histogram(value, histnorm='probability density', title = f"<b>Histogram of {mat}</b> shape : {value.shape} ", template="plotly_dark")
    fig.show()


q=0.8786615266594074

gamma2=0.7311313614476556

vx=0.980105194831573

R2=0.9843661140221802

sigma2=258.5383127472888



### samples simple discrete variables

In [4]:
# def sample_discrete(seed: int, values, probs, n_points: int):
#     return rv_discrete(seed = seed, values = (values, probs)).rvs(size=n_points)

# print(sample_discrete(seed: int, np.arange(3), , n_points: int))

# def sample_discrete_2dim(seed, dct_masse, n_points):
    
    

## 3. Sample from the conditional posterior of $z$

### Preliminary step: compute some random vectors

In [26]:
def compute_X_tilde(X,beta):
    """Compute T*s(z) \Tilde{X} matrix

    Args:
        X (np.array): T*p matrix of x predictors
        beta (np.array): 1*p vector of beta prior

    Returns:
        np.array: \Tilde{X} matrix
    """
    non_zero_beta=list(np.nonzero(beta)[0])
    X_tilde=X[:, non_zero_beta]
    return X_tilde

def compute_W_tilde(X_tilde, gamma2):
    """Compute s(z)*s(z) \Tilde{W} matrix

    Args:
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        gamma2 (float): gamma^2 

    Returns:
        np.array: \Tilde{W} matrix
    """
    I_s_z=np.identity(X_tilde.shape[1])
    return X_tilde.T@X_tilde+(1/gamma2)*I_s_z

def compute_Y_tilde(Y, U, phi):
    """Compute 1*T \Tilde{Y} matrix

    Args:
        Y (np.array): 1*T vector of target variables
        U (np.array): T*l matrix of u predictors
        phi (float): phi

    Returns:
        np.array: \Tilde{Y}
    """
    return Y-U@phi

def compute_estimator_beta_tilde(W_tilde, X_tilde, Y_tilde):
    """Compute 1*s(z) \hat{\Tilde{\beta}} vector

    Args:
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        X_tilde (np.array): T*s(z) \Tilde{X} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix

    Returns:
        np.array: 1*s(z) \hat{\Tilde{\beta}} vector
    """
    return np.linalg.inv(W_tilde)@X_tilde.T@Y_tilde

### Gibbs sampler

In [65]:
def compute_z_posterior(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T):
    """Compute \pi(z|Y, U, X, \phi, R^2, q) mass function

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples

    Returns:
        float: value of \pi(z|Y, U, X, \phi, R^2, q)
    """
    s_z=np.sum(Z)
    return q**s_z*(1-q)**(k-s_z)*(1/gamma2)**(s_z/2)*np.linalg.det(W_tilde)**(-0.5)*0.5**(-T/2)*(Y_tilde.T@Y_tilde-estimator_beta_tilde.T@W_tilde@estimator_beta_tilde)**(-T/2)*sp.special.gamma(T/2)


def compute_zi_posterior_conditional_z_excluded_i(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, zi, index):
    """Compute \pi(z_i|Y, X, U, \phi, R^2, q z_{-i}) for a specific zi

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        zi (int): Z_i variable
        index (int): index of Z_i variable in Z array

    Returns:
        float: value of \pi(z_i|Y, X, U, \phi, R^2, q z_{-i})
    """
    Z_i=np.copy(Z)
    Z_i[index]=zi
    z_posterior=compute_z_posterior(q=q, k=k, Z=Z_i, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    Z_0=np.copy(Z)
    Z_0[index]=0
    z_posterior_zi_equal_0=compute_z_posterior(q=q, k=k, Z=Z_0, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    Z_1=np.copy(Z)
    Z_1[index]=1
    z_posterior_zi_equal_1=compute_z_posterior(q=q, k=k, Z=Z_1, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T)
    return z_posterior/(z_posterior_zi_equal_0+z_posterior_zi_equal_1)

def simulated_zi_posterior_conditional_z_excluded_i(q, k, Z, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, index, seed):
    """Sample z_i|Y, X, U, \phi, R^2, q z_{-i} by inverse CDF method

    Args:
        q (float): q variable
        k (int): number of x predictors
        Z (np.array): 1*k array of z1,...,zk
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        index (int): index of Z_i variable in Z array
        seed (int): seed

    Returns:
        int: value of z_i|Y, X, U, \phi, R^2, q z_{-i} in {0,1} 
    """
    np.random.seed(seed)
    proba_success=compute_zi_posterior_conditional_z_excluded_i(q=q, k=k, Z=Z, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, zi=1, index=index)
    u=np.random.uniform(0,1)
    if u<1-proba_success:
        return 0
    else:
        return 1
    
def step_gibbs_sampler_z_posterior(q, k, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, seed, Z_t_minus_1):
    """One iteration of Gibbs sampler

    Args:
        q (float): q variable
        k (int): number of x predictors
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        seed (int): seed
        Z_t_minus_1 (_type_): Z_{t-1} variable sampled during the previous iteration (t-1 step)

    Returns:
        int: 
    """
    Z_t=np.copy(Z_t_minus_1)
    for i in range(len(Z_t)):
        Z_t[i]=simulated_zi_posterior_conditional_z_excluded_i(q=q, k=k, Z=Z_t, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, index=i, seed=seed)

    return Z_t

def gibbs_sampler_z(q, k, gamma2, W_tilde, Y_tilde, estimator_beta_tilde, T, seed, n_iter):
    """Gibbs sampler to simulate 1*k vector z|Y, U, X, \phi, R^2

    Args:
        q (float): q variable
        k (int): number of x predictors
        gamma2 (float): gamma^2 variable
        W_tilde (np.array): s(z)*s(z) \Tilde{W} matrix
        Y_tilde (np.array): 1*T \Tilde{Y} matrix
        estimator_beta_tilde (np.array): 1*s(z) \hat{\Tilde{\beta}} vector
        T (int): number of samples
        seed (int): seed
        n_iter (int): number of iterations

    Returns:
        np.array: z=(z_1,...,z_k) 1*k vector
    """
    Z_0=np.random.binomial(n=1, p=0.5, size=k)
    Z_t=Z_0
    for t in range(n_iter):
        Z_t=step_gibbs_sampler_z_posterior(q=q, k=k, gamma2=gamma2, W_tilde=W_tilde, Y_tilde=Y_tilde, estimator_beta_tilde=estimator_beta_tilde, T=T, seed=seed, Z_t_minus_1=Z_t)
    return Z_t

In [66]:
Z=dct['Z']
X=dct['X']
U=dct['U']
Y=dct['Y']
q=dct["q"]
gamma2=dct["gamma2"]
beta=dct["beta"]
phi=dct["phi"]

In [67]:
X_tilde=compute_X_tilde(X=X,beta=beta)
W_tilde=compute_W_tilde(X_tilde=X_tilde, gamma2=gamma2)
Y_tilde=compute_Y_tilde(Y=Y, U=U, phi=phi)
estimator_beta_tilde=compute_estimator_beta_tilde(W_tilde=W_tilde, X_tilde=X_tilde, Y_tilde=Y_tilde)

In [69]:
sample_posterior_z=gibbs_sampler_z(q=q,
                                   k=k, 
                                   gamma2=gamma2,
                                   W_tilde=W_tilde,
                                   Y_tilde=Y_tilde,
                                   estimator_beta_tilde=estimator_beta_tilde, 
                                   T=T,
                                   seed=seed,
                                   n_iter=10000)

print(sample_posterior_z)

C:\Users\yanis\AppData\Local\Temp\ipykernel_227272\3418378452.py:48: RuntimeWarning:

invalid value encountered in double_scalars



[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


* $l=0$ dans l'énoncé donc, $U=0$: bizarre comme certaines variances dépendent de U (celle du posterior de $\phi$) et valent donc 0
* Doute sur la génération de $\beta$ et $Z$: en théorie $\beta$, suit un mélange de lois (gaussien + dirac). Or, dans l'énoncé on demande explicitement de mettre $k-s$ élements de $\beta$ à 0 et les autres composantes suivent une normale centrée réduite. De plus, $Z_j$ suit une loi de bernouilli de param_tre $q$. Or, dans ce cas, il n'y a pas de simulation aléatoire de $Z$. On regarde juste les composantes de $\beta$ non-nulles. Je trouve ça étrange.